In [1]:
# 목차
# 1.
# logistic regression
# binary에 대한 cost를 자동으로 만들어주는 함수
# 전체 트레이닝 과정
# 2. 
# 평가
# 3.
# 클래스를 이용해서

# Logistic regression
- m by d -> m by 1 (0과1)
- sigmoid function -> 1/(1+e^(-X * W))
- 여기서 |X * W| = (m,d) * (d,1) = (m,1)

In [ ]:
# 필요한 것들
# 1. 도구 불러오기
# 2. 피쳐 데이터와 레이블 데이터 가져오기, tensor화 시키기
# 3. 예측 모델 W, b 만들기
# 4. 가설함수
# 5. loss 함수

In [4]:
import torch
import torch.nn as nn # 뉴럴 네트워크를 생성하기 위한 도구
import torch.nn.functional as F # loss function 쉽게 불러오기 위해서
import torch.optim as optim # optimizer 쉽게 만들기 위하여

In [5]:
torch.manual_seed(1)

In [6]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]] # (6,2) 데이터
y_data = [[0],[0],[0],[1],[1],[1]]

In [7]:
# 데이터를 토치텐서 형식으로 변환해줌
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [8]:
# Hypothesis 함수 만들기
W = torch.zeros((2,1), requires_grad=True)
# W가 2,1인 이유는 들어오는 데이터가 6*2이기 때문에 6*1 데이터로 만들기 위해서
b = torch.zeros(1, requires_grad=True)

In [9]:
hypothesis = 1/(1+torch.exp(-(x_train.matmul(W)+b)))

In [10]:
print(hypothesis)
print(hypothesis.shape)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [11]:
# 손실함수 (cost function) 구현
losses = -(y_train*torch.log(hypothesis)+
          (1-y_train)*torch.log(1-hypothesis))
print(losses)
cost = losses.mean()
print(cost)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)
tensor(0.6931, grad_fn=<MeanBackward0>)


# binary에 대한 cost를 자동으로 만들어주는 함수
- 앞에 한 것들을 한번에

In [12]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

# 전체 트레이닝 과정

In [13]:
# 1. 모델 W, b / optimizer
# 2. epoch 수 설정
# 3. epoch 동안 => (
#     (1) 가설함수, 손실함수 계산 / 
#     (2) cost를 통해서 gradient, 업데이트
#     (3) 일정 단위마다 결과 출력 (여기에서는 비용 출력 - cost.item())
#                )

In [14]:
W = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정 : SGD를 가지고 W와 b를 러닝레이트가 1인 상태로 훈련
optimizer = optim.SGD([W,b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # cost 계산
    hypothesis = 1/(1+torch.exp(-(x_train.matmul(W)+b)))
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 계산
    optimizer.zero_grad() # 기존에 혹시 gradient 구해 놓은것이 있으면 0으로
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print("Epoch {:4d}/{} Cost: {:.6f}".format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


# 평가

In [17]:
# accuracy = correct_prediction.sum().item() / len(correct_prediction)

In [18]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b) 
# 지금은 x_train 데이터가 들어있지만 x_test를 넣어야함
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


In [19]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [20]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


In [21]:
# 예시를 위해서 train 데이터로 했기 때문에 100%
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

The model has an accuracy of 100.00% for the training set.


# Higher implementation with class (실전)

In [14]:
import numpy as np

In [15]:
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [16]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8,1)  # {W, b}, (?by8)을 예측하는 모델
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [17]:
model = BinaryClassifier()

In [24]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range (nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = model(x_train)
    
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 10번마다 로그 출력
    if epoch % 10 ==0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print("Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%".format(
            epoch, nb_epochs, cost.item(), accuracy * 100
        ))
       
        

Epoch    0/100 Cost: 0.472452 Accuracy 77.08%
Epoch   10/100 Cost: 0.472388 Accuracy 77.21%
Epoch   20/100 Cost: 0.472331 Accuracy 77.21%
Epoch   30/100 Cost: 0.472278 Accuracy 77.21%
Epoch   40/100 Cost: 0.472231 Accuracy 77.08%
Epoch   50/100 Cost: 0.472188 Accuracy 77.08%
Epoch   60/100 Cost: 0.472148 Accuracy 77.08%
Epoch   70/100 Cost: 0.472112 Accuracy 77.08%
Epoch   80/100 Cost: 0.472079 Accuracy 77.08%
Epoch   90/100 Cost: 0.472048 Accuracy 77.08%
Epoch  100/100 Cost: 0.472020 Accuracy 77.08%


In [25]:
# 학습된 모델의 파라미터 값 확인하기
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.8913, -3.5399,  0.2260, -0.5833, -0.3301, -2.3799, -1.0032, -0.0717]],
       requires_grad=True)
Parameter containing:
tensor([0.1705], requires_grad=True)
